# A Machine Learning Model That Can Predict The Genre Of A Movie
###  [Dataset🗂📝](https://www.kaggle.com/datasets/hijest/genre-classification-dataset-imdb)

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score,precision_score
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn import set_config
set_config(display="diagram")
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

In [83]:
train_df = pd.read_csv("data/train_data.txt",sep=":::",header=None,engine="python")

In [84]:
train_df = train_df.drop(columns=[0],axis=1)

In [85]:
train_df.head()

,1,2,3
0,Oscar et la dame rose (2009),drama,Listening in to a conversation between his do...
1,Cupid (1997),thriller,A brother and sister with a past incestuous r...
2,"Young, Wild and Wonderful (1980)",adult,As the bus empties the students for their fie...
3,The Secret Sin (1915),drama,To help their unemployed father make ends mee...
4,The Unrecovered (2007),drama,The film's title refers not only to the un-re...


In [89]:
train_df.rename(columns={1:"movie",2:"genre",3:"overview"},inplace=True)

In [91]:
train_df.describe()

,movie,genre,overview
count,54214,54214,54214
unique,54214,27,54086
top,Oscar et la dame rose (2009),drama,Grammy - music award of the American academy ...
freq,1,13613,12


In [93]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54214 entries, 0 to 54213
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie     54214 non-null  object
 1   genre     54214 non-null  object
 2   overview  54214 non-null  object
dtypes: object(3)
memory usage: 1.2+ MB


In [95]:
train_df.isna().sum()

movie       0
genre       0
overview    0
dtype: int64

In [19]:
train_df.duplicated().sum()

0

In [99]:
train_df.genre = train_df.genre.apply(lambda x:x.lower())
train_df.overview = train_df.overview.apply(lambda x:x.lower())

In [101]:
train_df.head(5)

,movie,genre,overview
0,Oscar et la dame rose (2009),drama,listening in to a conversation between his do...
1,Cupid (1997),thriller,a brother and sister with a past incestuous r...
2,"Young, Wild and Wonderful (1980)",adult,as the bus empties the students for their fie...
3,The Secret Sin (1915),drama,to help their unemployed father make ends mee...
4,The Unrecovered (2007),drama,the film's title refers not only to the un-re...


### Remove WhiteSpace

In [26]:
def remove_whitespace(text):
    return text.str.strip()

In [28]:
train_df = train_df.apply(remove_whitespace)

In [30]:
train_df.head(5)

,movie,genre,overview
0,Oscar et la dame rose (2009),drama,listening in to a conversation between his doc...
1,Cupid (1997),thriller,a brother and sister with a past incestuous re...
2,"Young, Wild and Wonderful (1980)",adult,as the bus empties the students for their fiel...
3,The Secret Sin (1915),drama,to help their unemployed father make ends meet...
4,The Unrecovered (2007),drama,the film's title refers not only to the un-rec...


### Remove Punctuation

In [33]:
def remove_punctuation(text):
    return text.str.replace('[^\w\s]', '', regex=True)


In [35]:
train_df = train_df.apply(remove_punctuation)

### Word Tokenize

In [38]:
def word_token(text):
    return text.apply(lambda x: x.split())

In [40]:
train_df = train_df.apply(word_token)

In [42]:
train_df.head(5)

,movie,genre,overview
0,"[Oscar, et, la, dame, rose, 2009]",[drama],"[listening, in, to, a, conversation, between, ..."
1,"[Cupid, 1997]",[thriller],"[a, brother, and, sister, with, a, past, inces..."
2,"[Young, Wild, and, Wonderful, 1980]",[adult],"[as, the, bus, empties, the, students, for, th..."
3,"[The, Secret, Sin, 1915]",[drama],"[to, help, their, unemployed, father, make, en..."
4,"[The, Unrecovered, 2007]",[drama],"[the, films, title, refers, not, only, to, the..."


In [44]:
train_df["text"] = train_df.overview + train_df.movie

In [46]:
train_df = train_df[["genre","text"]]
train_df.head()

,genre,text
0,[drama],"[listening, in, to, a, conversation, between, ..."
1,[thriller],"[a, brother, and, sister, with, a, past, inces..."
2,[adult],"[as, the, bus, empties, the, students, for, th..."
3,[drama],"[to, help, their, unemployed, father, make, en..."
4,[drama],"[the, films, title, refers, not, only, to, the..."


### Remove StopWords

In [49]:
def remove_stopwords(text):
    return text.apply(lambda x: [word for word in x if word not in stopwords.words("english")])

In [51]:
train_df = train_df.apply(remove_stopwords)

In [53]:
lb = LabelEncoder()
genre_df  = train_df.genre
genre_df= genre_df.apply(lambda x: ' '.join(x))

genre_df = lb.fit_transform(genre_df)

In [55]:
genre_df

array([4, 7, 0, 4, 4, 3, 1, 2, 6, 5])

In [57]:
ps = PorterStemmer()

### Stemming

In [60]:
def stemming(text):
    return text.apply(lambda x: [ps.stem(word) for word in x ]) 

In [62]:
train_df = train_df.apply(stemming)

In [64]:
df = pd.concat([train_df,pd.DataFrame(genre_df)],axis=1)
df.rename(columns={0:"genre_lb"},inplace=True)

In [66]:
df['text'] = df['text'].apply(lambda x: ' '.join(x))
df['genre'] = df['genre'].apply(lambda x: ' '.join(x))

In [68]:
df.head()

,genre,text,genre_lb
0,drama,listen convers doctor parent 10yearold oscar l...,4
1,thriller,brother sister past incestu relationship curre...,7
2,adult,bu empti student field trip museum natur histo...,0
3,drama,help unemploy father make end meet edith twin ...,4
4,drama,film titl refer unrecov bodi ground zero also ...,4


### Export Clean Data

In [50]:
df.to_csv("data/clean_train_df.csv")

## Import Clean Data

In [73]:
new_train_df = pd.read_csv("data/clean_train_df.csv")

In [245]:
del new_train_df["Unnamed: 0"]

In [248]:
new_train_df.head(5)

,genre,text,genre_lb
0,drama,listen convers doctor parent 10yearold oscar l...,8
1,thriller,brother sister past incestu relationship curre...,24
2,adult,bu empti student field trip museum natur histo...,1
3,drama,help unemploy father make end meet edith twin ...,8
4,drama,film titl refer unrecov bodi ground zero also ...,8


### Vector Transformation

In [251]:
tfidf = TfidfVectorizer(max_features=6000)

In [253]:
x = tfidf.fit_transform(new_train_df.text).toarray()
x

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [263]:
y = new_train_df.genre_lb.values
y

array([ 8, 24,  1, ...,  7,  5, 12], dtype=int64)

In [265]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [266]:
mnb = MultinomialNB()
mnb.fit(x_train,y_train)

MultinomialNB()

### Prediction

In [268]:
y_pred = mnb.predict(x_test)
accuracy = accuracy_score(y_test,y_pred)
precision = precision_score(y_test,y_pred,average="macro")
print(f"accuracy {accuracy} , precision {precision}")
       

accuracy 0.49884718251406435 , precision 0.44762048726543713


C:\Users\RAJVEER\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Essential Data Preprocessing Steps for Reliable Pipeline Construction

In [107]:
df_pipe = pd.read_csv("data/train_data.txt",sep=":::",header=None,engine="python")

In [108]:
df_pipe.rename(columns={1:"movie",2:"genre",3:"overview"},inplace=True)

In [109]:
del df_pipe[0]

In [110]:
df_pipe["text"] = df_pipe.overview + df_pipe.movie
df_pipe=df_pipe[["genre","text"]]

In [459]:
drama = df_pipe[df_pipe.genre == " drama "][:500]
documentary =  df_pipe[df_pipe.genre ==" documentary "][:500]
comedy = df_pipe[df_pipe.genre ==" comedy "][:500]
short = df_pipe[df_pipe.genre ==" short "][:500]
thriller = df_pipe[df_pipe.genre ==" thriller "][:500]
action = df_pipe[df_pipe.genre ==" action "][:500]
horror = df_pipe[df_pipe.genre ==" horror "][:500]
western = df_pipe[df_pipe.genre ==" western "][:400]

In [461]:
df_ =df_pipe[~df_pipe.genre.isin([" horror "," drama ", " documentary ", " comedy ", " short ", " thriller ", " action ", " western "])]

In [463]:
df_pipe = pd.concat([drama,documentary,comedy,short,thriller,action,horror,western,df_])

In [465]:
x_ = df_pipe.drop(columns=["genre"])
y_ = df_pipe.genre

In [467]:
x_.head()

,text
0,Listening in to a conversation between his do...
3,To help their unemployed father make ends mee...
4,The film's title refers not only to the un-re...
11,"Danny, dying of Aids, returns home for his la..."
14,Amanda Wingfield dominates her children with ...


In [78]:
from scipy.sparse import csr_matrix

def sparse_to_csr(sparse_matrix):
    if isinstance(sparse_matrix, csr_matrix):
        return sparse_matrix
    return csr_matrix(sparse_matrix)

def string_fun(text):
    return text.astype(str).agg(''.join, axis=0)

In [473]:
x_train,x_test,y_train,y_test = train_test_split(x_,y_,test_size=0.1,random_state=4)
x_test.shape,x_train.shape

((1275, 1), (11468, 1))

## PipeLine

In [80]:
text_processing = Pipeline([
    ("remove_whitespace", FunctionTransformer(remove_whitespace, validate=False)),
    ('remove_punctuation', FunctionTransformer(remove_punctuation, validate=False)),
    ("word_token", FunctionTransformer(word_token, validate=False)),
    ("stopwords", FunctionTransformer(remove_stopwords, validate=False)),
    ("stemming", FunctionTransformer(stemming, validate=False)),
    ("string",FunctionTransformer(string_fun,validate=False)),
    ("TfidfVectorizer", TfidfVectorizer()),
    ('toarray', FunctionTransformer(sparse_to_csr, validate=False))  
])

tranform  = ColumnTransformer([
    ("vector", text_processing, "text")
])

model = MultinomialNB()


In [480]:
pipe = make_pipeline(tranform, model)

In [482]:
pipe.fit(x_train,y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('vector',
                                                  Pipeline(steps=[('remove_whitespace',
                                                                   FunctionTransformer(func=<function remove_whitespace at 0x000001A33D2453A0>)),
                                                                  ('remove_punctuation',
                                                                   FunctionTransformer(func=<function remove_punctuation at 0x000001A33706E2A0>)),
                                                                  ('word_token',
                                                                   FunctionTransformer(func=<function word_token at 0...
                                                                   FunctionTransformer(func=<function remove_stopwords at 0x000001A33A16ECA0>)),
                                                                  ('stemming',
                                                                   FunctionTransformer(func=<function stemming at 0x000001A33706F920>)),
                                                                  ('string',
                                                                   FunctionTransformer(func=<function string_fun at 0x000001A31FD523E0>)),
                                                                  ('TfidfVectorizer',
                                                                   TfidfVectorizer()),
                                                                  ('toarray',
                                                                   FunctionTransformer(func=<function sparse_to_csr at 0x000001A31AD8C040>))]),
                                                  'text')])),
                ('multinomialnb', MultinomialNB())])

### Export Pipeline 

In [557]:
import dill

In [559]:
dill.dump(pipe,open("pipe1.pkl","wb"))

In [485]:
accuracy_score(y_test, pipe.predict(x_test))

0.30823529411764705